In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
import re

# --- 1. Chargement des Données (notre base de connaissances) ---
try:
    df = pd.read_csv('C:/Users/Infinix/Desktop/Projet fil rouge/data/processed/medical_dataset_ml.csv')
except FileNotFoundError:
    print("Erreur: Fichier 'data/processed/medical_dataset_ml.csv' non trouvé.")
    exit()

# Nettoyage (important pour que les phrases aient du sens)
def clean_symptoms_for_nlp(text):
    if not isinstance(text, str):
        return ""
    # Transforme "itching,skin_rash" en "itching skin rash"
    text = re.sub(r'[_,]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['symptoms_processed'] = df['All_Symptoms'].apply(clean_symptoms_for_nlp)
print(f"Base de connaissances chargée : {df.shape[0]} cas.")

# --- 2. Chargement du "Cerveau" (Modèle Sémantique) ---
# 'paraphrase-multilingual-MiniLM-L12-v2' est excellent pour débuter :
# Il est léger, rapide et comprend plusieurs langues (dont le français et l'anglais)
# Plus tard, on pourra le remplacer par un modèle médical (ex: 'BioBERT')
print("Chargement du modèle SentenceTransformer...")
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print("Modèle chargé.")

# --- 3. Vectorisation de la Base de Connaissances ---
# C'est l'étape la plus longue, mais on ne la fait qu'une seule fois.
corpus_symptoms = df['symptoms_processed'].tolist()
# On crée les embeddings (vecteurs) pour chaque liste de symptômes
# Cela peut prendre quelques minutes
print("Création des embeddings pour la base de connaissances...")
corpus_embeddings = model.encode(corpus_symptoms, convert_to_tensor=True, show_progress_bar=True)
print(f"Base de connaissances vectorisée (Shape: {corpus_embeddings.shape})")

# --- 4. Simulation d'un Diagnostic (Recherche Sémantique) ---
print("\n--- Simulation d'une Consultation MedAnalyzer AI ---")

# Essayez de changer cette phrase !
# Mettez des fautes, utilisez du français, etc.
#query = "My patient is itching a lot and has red skin rashes and some strange patches."
query = "J'ai des douleurs articulaires et de la fatigue chronique."

print(f"Requête du médecin : \"{query}\"")

# Vectorisation de la requête
query_embedding = model.encode(query, convert_to_tensor=True)

# --- 5. Calcul de la Similarité (Le "Diagnostic") ---
# Nous utilisons la similarité cosinus pour trouver la phrase la plus proche dans notre corpus
# 'util.semantic_search' fait tout le travail pour nous
search_results = util.semantic_search(query_embedding, corpus_embeddings, top_k=3)

print("\n--- Proposition de Diagnostic Différentiel ---")

# search_results[0] contient les résultats pour la première (et unique) requête
for i, result in enumerate(search_results[0]):
    corpus_index = result['corpus_id']
    similarity_score = result['score']
    
    # On récupère la ligne correspondante dans notre DataFrame
    matched_case = df.iloc[corpus_index]
    disease = matched_case['Disease']
    
    print(f"  {i+1}. Diagnostic : **{disease}**")
    print(f"     Score de Confiance (Similarité) : {similarity_score * 100:.2f} %")
    print(f"     Symptômes du cas similaire : \"{matched_case['symptoms_processed']}\"")
    print("-" * 20)

Base de connaissances chargée : 4920 cas.
Chargement du modèle SentenceTransformer...
Modèle chargé.
Création des embeddings pour la base de connaissances...


Batches: 100%|██████████| 154/154 [01:16<00:00,  2.02it/s]

Base de connaissances vectorisée (Shape: torch.Size([4920, 384]))

--- Simulation d'une Consultation MedAnalyzer AI ---
Requête du médecin : "J'ai des douleurs articulaires et de la fatigue chronique."

--- Proposition de Diagnostic Différentiel ---
  1. Diagnostic : **Cervical spondylosis**
     Score de Confiance (Similarité) : 62.80 %
     Symptômes du cas similaire : "back pain weakness in limbs dizziness loss of balance 6"
--------------------
  2. Diagnostic : **Cervical spondylosis**
     Score de Confiance (Similarité) : 62.80 %
     Symptômes du cas similaire : "back pain weakness in limbs dizziness loss of balance 6"
--------------------
  3. Diagnostic : **Cervical spondylosis**
     Score de Confiance (Similarité) : 62.80 %
     Symptômes du cas similaire : "back pain weakness in limbs dizziness loss of balance 6"
--------------------


In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
import time

# --- 1. CHARGEMENT (RAPIDE) ---
print("Démarrage de l'application de diagnostic...")

# Charger le modèle (rapide, depuis le cache)
print("Chargement du modèle SentenceTransformer...")
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print("Modèle chargé.")

# Charger les données (rapide, depuis les fichiers pré-calculés)
print("Chargement de la base de connaissances pré-calculée...")
try:
    df = pd.read_pickle("C:/Users/Infinix/Desktop/Projet fil rouge/processed_dataframe.pkl")
    corpus_embeddings = torch.load("C:/Users/Infinix/Desktop/Projet fil rouge/corpus_embeddings.pt")
except FileNotFoundError:
    print("Erreur: Fichiers 'processed_dataframe.pkl' ou 'corpus_embeddings.pt' non trouvés.")
    print("Veuillez d'abord lancer le script 'create_embeddings.py'.")
    exit()

print(f"Base de connaissances chargée (Shape: {corpus_embeddings.shape}). Prêt à diagnostiquer.")


# --- 4. Simulation d'un Diagnostic (Recherche Sémantique) ---
print("\n--- Simulation d'une Consultation MedAnalyzer AI ---")

query = "My patient is itching a lot and has red skin rashes and some strange patches."
print(f"Requête du médecin : \"{query}\"")

# --- 5. Calcul de la Similarité (Le "Diagnostic" RAPIDE) ---
print("Calcul de la prédiction...")
start_time = time.time()

# Étape 1 : Encoder la requête (quelques millisecondes)
query_embedding = model.encode(query, convert_to_tensor=True)

# Étape 2 : Chercher (quelques millisecondes)
search_results = util.semantic_search(query_embedding, corpus_embeddings, top_k=3)

end_time = time.time()
print(f"Prédiction terminée en {end_time - start_time:.4f} secondes.") # Tu verras que c'est < 0.1s

print("\n--- Proposition de Diagnostic Différentiel ---")

for i, result in enumerate(search_results[0]):
    corpus_index = result['corpus_id']
    similarity_score = result['score']
    
    matched_case = df.iloc[corpus_index]
    disease = matched_case['Disease']
    
    print(f"  {i+1}. Diagnostic : **{disease}**")
    print(f"     Score de Confiance (Similarité) : {similarity_score * 100:.2f} %")
    print(f"     Symptômes du cas similaire : \"{matched_case['symptoms_processed']}\"")
    print("-" * 20)

Démarrage de l'application de diagnostic...
Chargement du modèle SentenceTransformer...
Modèle chargé.
Chargement de la base de connaissances pré-calculée...
Base de connaissances chargée (Shape: torch.Size([4920, 384])). Prêt à diagnostiquer.

--- Simulation d'une Consultation MedAnalyzer AI ---
Requête du médecin : "My patient is itching a lot and has red skin rashes and some strange patches."
Calcul de la prédiction...
Prédiction terminée en 0.0627 secondes.

--- Proposition de Diagnostic Différentiel ---
  1. Diagnostic : **Fungal infection**
     Score de Confiance (Similarité) : 77.64 %
     Symptômes du cas similaire : "itching skin rash dischromic patches 5"
--------------------
  2. Diagnostic : **Fungal infection**
     Score de Confiance (Similarité) : 77.64 %
     Symptômes du cas similaire : "itching skin rash dischromic patches 5"
--------------------
  3. Diagnostic : **Fungal infection**
     Score de Confiance (Similarité) : 77.64 %
     Symptômes du cas similaire : "i